# **NLP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Mathematical Formulation:

Parameters:
- p_i = prices for each of the operational costs
- C = Cost to operate the system
- R = Revenue from the system performance
- r = selling price per barrel of alkylate per motor octane
- r_i = cost rate for each operational item 

Decision Variables:
- x1 = olefin feed (barrels per day)
- x2 = isobutane recycle (barrels per day)
- x3 = acid addition rate (thousands of pounds per day)
- x4 = alkylate yield (barrels per day)
- x5 = isobutane makeup (barrels per day)
- x6 = acid strength (weight per cent)
- x7 = motor octane number
- x8 = external isobutane-to-olefin ratio
- x9 = acid dilution factor
- x10 = F-4 performance number

Objective Function:
Maximize profit = Revenue - Costs
Max P = r*x4*x7 - sum(p_i*x_i) 

Constraints:
- Statistical limit of variables: L_i <= x_i <= U_i 
- External isobutane-to-olefin ratio: x8 = (x2 + x5) / x1
- Acid Strength: x6 = (93000 * x3) / (x3 * x9 + 1000 * x3)
- Alkylate yield: 1.22 * x4 = x1 + x5 
- Alkylate yield

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here it is. Note that we define the example data here as the prices for the operational costs.

```python
from pyomo.environ import *

# price for each operational costs
sample_prices = [500, 300, 240, 420, 520, 100, 230, 600, 240, 400]  # Sample data

# Create a Concrete Model as a container
model = ConcreteModel()

# Define variables
model.x = Var(range(1, 11), domain=NonNegativeReals)

# Define the objective Rule
def obj_rule(m):
    return r*m.x[4]*m.x[7] - sum(sample_prices[i] * m.x[i+1] for i in range(10))

# Add objective
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
model.cons = ConstraintList()

model.cons.add(model.x[8] == (model.x[2] + model.x[5]) / model.x[1])
model.cons.add(model.x[6] == (93000 * model.x[3]) / (model.x[3] * model.x[9] + 1000 * model.x[3]))
model.cons.add(1.22 * model.x[4] == model.x[1] + model.x[5])
model.cons.add(model.x[4] == model.x[1] * (1.12 + 0.13167 * model.x[8] -0.00667 * m.x[8]**2))
model.cons.add(model.x[7] ==  86.35 + 1.098 * model.x[8] - 0.038 * model.x[8]**2 + 0.325 * (model.x[6] - 89))
model.cons.add(model.x[9] == 35.82 - 0.222 * model.x[10])
model.cons.add(model.x[10] == -133 + 3 * model.x[7])

# Add bounds
for i in range(10):
    model.x[i+1].setlb(L[i])  # Lower bound
    model.x[i+1].setub(U[i])  # Upper bound

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Display all variable information
model.pprint()
```
Please replace L[i] and U[i] with the actual lower and upper bounds for each variable. The price per barrel of alkylate per motor octane (r) is not given in the problem statement, you need to provide this value. In the place of 'ipopt' write the solver that you're going to use to solve this optimization problem.

### **Run the code Generated by GPT4**

In [3]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.7 MB/s eta 0:00:00


In [ ]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# price for each operational costs
sample_prices = [500, 300, 240, 420, 520, 100, 230, 600, 240, 400]  # Sample data

# Create a Concrete Model as a container
model = ConcreteModel()

# Define variables
model.x = Var(range(1, 11), domain=NonNegativeReals)

# Define the objective Rule
def obj_rule(m):
    return r*m.x[4]*m.x[7] - sum(sample_prices[i] * m.x[i+1] for i in range(10))

# Add objective
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
model.cons = ConstraintList()

model.cons.add(model.x[8] == (model.x[2] + model.x[5]) / model.x[1])
model.cons.add(model.x[6] == (93000 * model.x[3]) / (model.x[3] * model.x[9] + 1000 * model.x[3]))
model.cons.add(1.22 * model.x[4] == model.x[1] + model.x[5])
model.cons.add(model.x[4] == model.x[1] * (1.12 + 0.13167 * model.x[8] -0.00667 * m.x[8]**2))
model.cons.add(model.x[7] ==  86.35 + 1.098 * model.x[8] - 0.038 * model.x[8]**2 + 0.325 * (model.x[6] - 89))
model.cons.add(model.x[9] == 35.82 - 0.222 * model.x[10])
model.cons.add(model.x[10] == -133 + 3 * model.x[7])

# Add bounds
for i in range(10):
    model.x[i+1].setlb(L[i])  # Lower bound
    model.x[i+1].setub(U[i])  # Upper bound

# Solve the model
solver = SolverFactory('knitro')
solver.solve(model)

# Display all variable information
model.pprint()

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
NameError: name 'r' is not defined
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    NameError: name 'r' is not defined


NameError: name 'r' is not defined

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [5]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [8]:
from pyomo.environ import *

# price for each operational costs
sample_prices = [5.04, 0.035, 10.0, 3.36]  # Sample data

L = [0, 0, 0, 0, 0, 85, 90, 3, 1.2, 145]
U = [2000, 16000, 120, 5000, 2000, 93, 95, 12, 4, 162]

r =  0.63
# Create a Concrete Model as a container
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)
model.x4 = Var(domain=NonNegativeReals)
model.x5 = Var(domain=NonNegativeReals)
model.x6 = Var(domain=NonNegativeReals)
model.x7 = Var(domain=NonNegativeReals)
model.x8 = Var(domain=NonNegativeReals)
model.x9 = Var(domain=NonNegativeReals)
model.x10 = Var(domain=NonNegativeReals)

# Define the objective Rule
def obj_rule(m):
    return r*model.x4*model.x7 - sample_prices[0]*model.x1 -  sample_prices[1]*model.x2 - sample_prices[2]*model.x3 - sample_prices[3]*model.x5

# Add objective
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
model.cons = ConstraintList()

model.cons.add(model.x8 == (model.x2 + model.x5) / model.x1)
model.cons.add(model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3))
model.cons.add(1.22 * model.x4 == model.x1 + model.x5)
model.cons.add(model.x4 == model.x1 * (1.12 + 0.13167 * model.x8 -0.00667 * model.x8**2))
model.cons.add(model.x7 ==  86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89))
model.cons.add(model.x9 == 35.82 - 0.222 * model.x10)
model.cons.add(model.x10 == -133 + 3 * model.x7)

# Add bounds
variables = [model.x1, model.x2, model.x3, model.x4, model.x5, model.x6, model.x7, model.x8, model.x9, model.x10]
for i in range(10):
    model.cons.add(L[i] <= variables[i])
    model.cons.add(variables[i] <= U[i])

# Solve the model
solver = SolverFactory('couenne')
solver.solve(model)

# Display all variable information
model.pprint()
print(model.obj())

10 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1966.7580859574266 :  None : False : False : NonNegativeReals
    x10 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 145.0 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 9188.898729326876 :  None : False : False : NonNegativeReals
    x3 : Size=1, Index=None
        Key  : Lower : Value                  : Upper : Fixed : Stale : Domain
        None :     0 : 4.2523237497050463e-13 :  None : False : False : NonNegativeReals
    x4 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 3251.441054063465 :  None : False : False : NonNegativeReals
    x5 : Size=1, Index=None
        Key  : Lower : Value  : U